<a href="https://colab.research.google.com/github/GBH7/Meachine-Learning/blob/main/Australia_house_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

In [27]:
data = pd.read_csv('RealEstateAU_1000_Samples.csv') 

In [8]:
data.head()

,index,TID,breadcrumb,category_name,property_type,building_size,land_size,preferred_size,open_date,listing_agency,...,state,zip_code,phone,latitude,longitude,product_depth,bedroom_count,bathroom_count,parking_count,RunDate
0,0,1350988,Buy>NT>DARWIN CITY,Real Estate & Property for sale in DARWIN CITY...,House,NaN,NaN,NaN,Added 2 hours ago,Professionals - DARWIN CITY,...,NT,800,08 8941 8289,NaN,NaN,premiere,2.0,1.0,1.0,2022-05-27 15:54:05
1,1,1350989,Buy>NT>DARWIN CITY,Real Estate & Property for sale in DARWIN CITY...,Apartment,171m²,NaN,171m²,Added 7 hours ago,Nick Mousellis Real Estate - Eview Group Member,...,NT,800,0411724000,NaN,NaN,premiere,3.0,2.0,2.0,2022-05-27 15:54:05
2,2,1350990,Buy>NT>DARWIN CITY,Real Estate & Property for sale in DARWIN CITY...,Unit,NaN,NaN,NaN,Added 22 hours ago,Habitat Real Estate - THE GARDENS,...,NT,800,08 8981 0080,NaN,NaN,premiere,2.0,1.0,1.0,2022-05-27 15:54:05
3,3,1350991,Buy>NT>DARWIN CITY,Real Estate & Property for sale in DARWIN CITY...,House,NaN,NaN,NaN,Added yesterday,Ray White - NIGHTCLIFF,...,NT,800,08 8982 2403,NaN,NaN,premiere,1.0,1.0,0.0,2022-05-27 15:54:05
4,4,1350992,Buy>NT>DARWIN CITY,Real Estate & Property for sale in DARWIN CITY...,Unit,201m²,NaN,201m²,Added yesterday,Carol Need Real Estate - Fannie Bay,...,NT,800,0418885966,NaN,NaN,premiere,3.0,2.0,2.0,2022-05-27 15:54:05


In [28]:
data = data[['property_type','building_size', 'land_size', 'location_name', 'preferred_size','zip_code',
             'product_depth','bedroom_count','bathroom_count','parking_count', 'price']]

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   property_type   1000 non-null   object 
 1   building_size   280 non-null    object 
 2   land_size       533 non-null    object 
 3   location_name   1000 non-null   object 
 4   preferred_size  609 non-null    object 
 5   zip_code        1000 non-null   int64  
 6   product_depth   1000 non-null   object 
 7   bedroom_count   967 non-null    float64
 8   bathroom_count  967 non-null    float64
 9   parking_count   967 non-null    float64
 10  price           1000 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 86.1+ KB


In [11]:
data.describe()

,zip_code,bedroom_count,bathroom_count,parking_count
count,1000.00000,967.000000,967.000000,967.000000
mean,816.64600,2.866598,1.739400,2.152017
std,13.22057,1.151914,0.635663,1.514818
min,800.00000,0.000000,1.000000,0.000000
25%,800.00000,2.000000,1.000000,1.000000
50%,820.00000,3.000000,2.000000,2.000000
75%,830.00000,4.000000,2.000000,2.000000
max,839.00000,9.000000,5.000000,12.000000


In [36]:
#결측치 0으로 처리
data.fillna(0,inplace = True)

In [30]:
data['price'] = data['price'].str.replace(pat=r'[^\w]', repl=r'', regex=True)
data['price'] = data['price'].str.extract(r'(\d+)')

data['building_size'] = data['building_size'].str.replace(pat=r'[^\w]', repl=r'', regex=True)
data['building_size'] = data['building_size'].str.extract(r'(\d+)')

data['location_name'] = data['location_name'].str.replace(pat=r'[^\w]', repl=r'', regex=True)
data['location_name'] = data['location_name'].str.extract(r'(\d+)')

data['preferred_size'] = data['preferred_size'].str.replace(pat=r'[^\w]', repl=r'', regex=True)
data['preferred_size'] = data['preferred_size'].str.extract(r'(\d+)')

data['land_size'] = data['land_size'].str.replace(pat=r'[^\w]', repl=r'', regex=True)
data['land_size'] = data['land_size'].str.extract(r'(\d+)')

In [31]:
#'bedroom_count','bathroom_count','parking_count'를 실수에서 정수로 변환
data[['bedroom_count','bathroom_count','parking_count']] = data[['bedroom_count','bathroom_count','parking_count']].astype('int64')

In [32]:
#product_depth를 더미변환
depth = pd.get_dummies(data['product_depth'])
data = pd.concat([data, depth], axis=1)
data.drop('product_depth', axis=1, inplace=True)

In [33]:
#property_type 더미변환
propertytype = pd.get_dummies(data['property_type'])
data = pd.concat([data, propertytype], axis=1)
data.drop('property_type', axis=1, inplace=True)

In [38]:
#결측치 0으로 처리
data.fillna(0,inplace = True)

In [37]:
data[['building_size','land_size','location_name','preferred_size','price']] = data[['building_size',
                                                                                     'land_size','location_name',
                                                                                     'preferred_size','price']].astype('int64')

In [39]:
#이상치 처리
colL  = ['building_size','land_size', 'location_name', 'preferred_size', 'zip_code', 'bedroom_count', 'bathroom_count',
      'parking_count','price']

train_iqr = data.copy()
for col in colL:
  Q1 = train_iqr[col].quantile(.25)
  Q3 = train_iqr[col].quantile(.75)
  IQR = Q3 - Q1
  train_del = train_iqr[((Q1-1.5*IQR) > train_iqr[col]) | ((Q3+1.5*IQR) < train_iqr[col])]
  train_iqr = train_iqr.drop(train_del.index, axis=0)
train_iqr

,building_size,land_size,location_name,preferred_size,zip_code,bedroom_count,bathroom_count,parking_count,price,feature,...,Duplex/Semi-detached,House,Lifestyle,Other,Residential Land,Studio,Townhouse,Unit,Villa,Warehouse
0,0,0,435000,0,800,2,1,1,435000,0,...,0,1,0,0,0,0,0,0,0,0
1,171,0,320000,171,800,3,2,2,320000,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,310000,0,800,2,1,1,310000,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,259000,0,800,1,1,0,259000,0,...,0,1,0,0,0,0,0,0,0,0
6,0,0,820000,0,800,3,2,2,820000,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,0,0,270000,0,800,1,1,1,270000,1,...,0,0,0,0,0,0,0,1,0,0
992,0,0,390000,0,820,2,2,2,390000,1,...,0,0,0,0,0,0,0,1,0,0
993,0,0,495000,0,830,4,2,2,495000,1,...,0,1,0,0,0,0,0,0,0,0
994,0,0,0,0,820,2,1,2,0,0,...,0,0,0,0,0,0,0,1,0,0


In [40]:
#train 데이터 내에서 트레이닝, 테스트 데이터셋 분리함.
from sklearn.model_selection import train_test_split
train_x = train_iqr.drop(['price'],axis = 1)
train_y = train_iqr['price']
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.3, random_state=42)

In [41]:
#X_train 데이터 표준화
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)

In [42]:
#X_test 데이터 표준화
X_test=scaler.transform(X_test)

In [43]:
#AdaBoost
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=5), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5)
ada_clf.fit(X_train,y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=200)

In [44]:
#rmse 평가
from pandas.core.nanops import F
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score, precision_score, recall_score, confusion_matrix

pred_ada = ada_clf.predict(X_test)

mse = mean_squared_error(y_test, pred_ada)
rmse = np.sqrt(mse)
print(rmse)

5260.0277714915865


In [45]:
#accuracy 평가
accuracy_score(pred_ada,y_test)

0.8186274509803921